<a href="https://colab.research.google.com/github/marcusnk237/Gradcam_plus_plus/blob/main/Gradcam_plus_plus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Libraries

In [ ]:
!pip install tensorflow_addons
import cv2
import gc
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow.compat.v1.keras.backend as K
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.collections as mcoll
import matplotlib as mpl
mpl.style.use('seaborn')
import numpy as np
import itertools
import logging
# Set random seed
np.random.seed(123)

## Grad-CAM ++ Function

##### Output image : The output will be an image, with the signal as the foreground, and the heatmap as the background

In [ ]:
def compute_cam_image_output (model, data , layer_name , W , H):
        """
        model: The Deep Learning model
        data : A input data
        layer_name : The target layer for explanation
        W : Width of the result heatmap
        H : Height of the result heatmap
        """
        # input layer, model output layer and target layer
        grad_model = tf.keras.models.Model(inputs=[model.inputs],outputs=[model.get_layer(layer_name).output, model.output])     
        
        # Getting gradients of input layer, model output layer (predictions) and target layer
        with tf.GradientTape() as tape:
            inputs = np.expand_dims(data,axis=0)
            conv_outs, predictions = grad_model(inputs) 
            class_idx = tf.argmax(predictions[0])
            y_c = predictions[:, class_idx]

        batch_grads = tape.gradient(y_c, conv_outs) 
        grads = batch_grads[0]
        
        # First, second and third derivative of output gradient
        first = tf.exp(y_c) * grads
        second = tf.exp(y_c) * tf.pow(grads, 2)
        third = tf.exp(y_c) * tf.pow(grads, 3)
        
        # Compute salienty maps for the class_idx prediction
        global_sum = tf.reduce_sum(tf.reshape(conv_outs[0], shape=(-1, first.shape[1])), axis=0)
        alpha_num = second
        alpha_denom = second * 2.0 + third * tf.reshape(global_sum, shape=(1,1,first.shape[1]))
        alpha_denom = tf.where(alpha_denom != 0.0, alpha_denom, tf.ones(shape=alpha_denom.shape))
        alphas = alpha_num / alpha_denom
        weights = tf.maximum(first, 0.0)
        alpha_normalization_constant = tf.reduce_sum(tf.reduce_sum(alphas, axis=0), axis=0)
        alphas /= tf.reshape(alpha_normalization_constant, shape=(1,1,first.shape[1]))
        alphas_thresholding = np.where(weights, alphas, 0.0)

        alpha_normalization_constant = tf.reduce_sum(tf.reduce_sum(alphas_thresholding, axis=0),axis=0)
        alpha_normalization_constant_processed = tf.where(alpha_normalization_constant != 0.0, alpha_normalization_constant,
                                                          tf.ones(alpha_normalization_constant.shape))

        alphas /= tf.reshape(alpha_normalization_constant_processed, shape=(1,1,first.shape[1]))
        deep_linearization_weights = tf.reduce_sum(tf.reshape((weights*alphas), shape=(-1,first.shape[1])), axis=0)
        grad_CAM_map = tf.reduce_sum(deep_linearization_weights * conv_outs[0], axis=-1)
        
        # Normalization
        cam = np.maximum(grad_CAM_map, 0)
        cam = cam / np.max(cam)  
        
        # Turn result into a heatmap
        heatmap=[]
        heatmap.append(cam.tolist())
        big_heatmap = cv2.resize(np.array(heatmap), dsize=(W, H),interpolation=cv2.INTER_CUBIC)

        # Plotting input data and the Grad-CAM ++ results
        plt.imshow(big_heatmap, cmap='rainbow', aspect="auto", interpolation='nearest',extent=[0,len(data)-1,(-1*H),H], alpha=0.9)
        plt.title(f"Grad-CAM ++ Visualization")
        plt.plot(data*80,'k')
        plt.grid(False)
        plt.colorbar()
        plt.show()

##### Output signal : The output will be the signal with each important segment draw with a color which represent the importance for the classification/prediction

In [ ]:
def multicolored_lines(x,y,heatmap,title_name):
    fig, ax = plt.subplots()
    lc = colorline(x, y, heatmap,cmap='rainbow')
    plt.colorbar(lc)
    lc.set_linewidth(2)
    lc.set_alpha(0.8)
    plt.xlim(x.min(), x.max())
    plt.ylim(y.min(), y.max())
    plt.title(title_name)
    plt.grid(False)
    plt.show()

def colorline(x, y, heatmap,cmap='rainbow'):
    z = np.array(heatmap)
    points = np.array([x, y]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)
    lc = mcoll.LineCollection(segments, array=z, cmap=cmap)
    ax = plt.gca()
    ax.add_collection(lc)
    return lc


In [ ]:
def compute_cam_1d_output (model, data , layer_name , N):
        """
        model: The Deep Learning model
        data : A input data
        layer_name : The target layer for explanation
        W : Width of the result heatmap
        H : Height of the result heatmap
        """
        # input layer, model output layer and target layer
        grad_model = tf.keras.models.Model(inputs=[model.inputs],outputs=[model.get_layer(layer_name).output, model.output])     
        
        # Getting gradients of input layer, model output layer (predictions) and target layer
        with tf.GradientTape() as tape:
            inputs = np.expand_dims(data,axis=0)
            conv_outs, predictions = grad_model(inputs) 
            class_idx = tf.argmax(predictions[0])
            y_c = predictions[:, class_idx]

        batch_grads = tape.gradient(y_c, conv_outs) 
        grads = batch_grads[0]
        
        # First, second and third derivative of output gradient
        first = tf.exp(y_c) * grads
        second = tf.exp(y_c) * tf.pow(grads, 2)
        third = tf.exp(y_c) * tf.pow(grads, 3)
        
        # Compute salienty maps for the class_idx prediction
        global_sum = tf.reduce_sum(tf.reshape(conv_outs[0], shape=(-1, first.shape[1])), axis=0)
        alpha_num = second
        alpha_denom = second * 2.0 + third * tf.reshape(global_sum, shape=(1,1,first.shape[1]))
        alpha_denom = tf.where(alpha_denom != 0.0, alpha_denom, tf.ones(shape=alpha_denom.shape))
        alphas = alpha_num / alpha_denom
        weights = tf.maximum(first, 0.0)
        alpha_normalization_constant = tf.reduce_sum(tf.reduce_sum(alphas, axis=0), axis=0)
        alphas /= tf.reshape(alpha_normalization_constant, shape=(1,1,first.shape[1]))
        alphas_thresholding = np.where(weights, alphas, 0.0)

        alpha_normalization_constant = tf.reduce_sum(tf.reduce_sum(alphas_thresholding, axis=0),axis=0)
        alpha_normalization_constant_processed = tf.where(alpha_normalization_constant != 0.0, alpha_normalization_constant,
                                                          tf.ones(alpha_normalization_constant.shape))

        alphas /= tf.reshape(alpha_normalization_constant_processed, shape=(1,1,first.shape[1]))
        deep_linearization_weights = tf.reduce_sum(tf.reshape((weights*alphas), shape=(-1,first.shape[1])), axis=0)
        grad_CAM_map = tf.reduce_sum(deep_linearization_weights * conv_outs[0], axis=-1)
        
        # Normalization
        cam = np.maximum(grad_CAM_map, 0)
        cam = cam / np.max(cam)  
        
        # Turn result into a heatmap
        heatmap=[]
        heatmap.append(cam.tolist())
        big_heatmap = cv2.resize(np.array(heatmap), dsize=(W, H),interpolation=cv2.INTER_CUBIC)
        x = np.linspace(0, N, len(data))
        multicolored_lines(x,data[:, 0],big_heatmap[0],f"GradCAM ++ Visualization")